In [1]:
import numpy as np
import tifffile
from utilities import two_point_correlation
import pandas as pd
from tqdm import tnrange

## File Directories

We specify here the file directories and endings for each of the images.  
This allows all computations to be made within one notebook and reduces number of necessary notebooks.

In [2]:
#strings to output and input locations
beadpack_dic = {
    "orig_input": "../../../data/beadpack/original/tiff/beadpack.tif",
    "synthetic_input": "../../../data/beadpack/sampled/postprocessed/dcgan_200_epoch_210/beadpack_200_",
    "synthetic_ending": ".tiff",
    "out_direc": "../../../analysis/covariance/beadpack/",
    "seed_min": 43,
    "seed_max": 64
}

berea_dic = {
    "orig_input": "../../../data/berea/original/raw/berea.tif",
    "synthetic_input": "../../../data/berea/sampled/postprocessed/tiff/dcgan_200/berea_generated_",
    "synthetic_ending": "_crop.tiff",
    "out_direc": "../../../analysis/covariance/berea/",
    "seed_min": 43,
    "seed_max": 64
}

ketton_dic = {
    "orig_input": "../../../data/ketton/original/raw/ketton.tif",
    "synthetic_input": "../../../data/ketton/sampled/postprocessed/tiff/200/ketton_desp_64_",
    "synthetic_ending": ".tiff",
    "out_direc": "../../../analysis/covariance/ketton/",
    "seed_min": 43,
    "seed_max": 64
}

In [3]:
data_dic = ketton_dic
orig_img = tifffile.imread(data_dic["orig_input"])
out_direc = data_dic["out_direc"]
synthetic_input = data_dic["synthetic_input"]
synthetic_ending = data_dic["synthetic_ending"]
seed_min, seed_max = data_dic["seed_min"], data_dic["seed_max"]

## Data Loading

We load the dataset of the original segmented training image and get the largest and smallest value in the dataset.  
This is necessary due to different images may be [0-1] or [0-255] segmented.  
Therefore we make this check before hand to compute for the correct phase.

In [4]:
pore_phase = orig_img.min()
grain_phase = orig_img.max()
print "Pore Phase Value: ", pore_phase
print "Grain Phase Value: ", grain_phase

Pore Phase Value:  0
Grain Phase Value:  255


## Covariance Computation - Pore Phase

We make use of a helper function in utilities.py to compute the covariance in each cartesian direction.  
The resulting dataset is a cube of the average counts at each distance $r$ for each value in the plane of computation.

In [5]:
two_point_covariance_pore_phase_orig = {}
for i, direc in enumerate(["x", "y", "z"]):
    two_point_direc = two_point_correlation(orig_img, i, var=pore_phase)
    two_point_covariance_pore_phase_orig[direc] = two_point_direc

We now average in the first two dimensions e.g. covariance in x-direction, average across y, z directions  
to obtain the directional covariance. Perform this for all three directions.

In [6]:
direc_covariances_pore_phase_orig = {}
for direc in ["x", "y", "z"]:
    direc_covariances_pore_phase_orig[direc] = np.mean(np.mean(two_point_covariance_pore_phase_orig[direc], axis=0), axis=0)
print direc_covariances_pore_phase_orig["x"].shape

(256L,)


We store the computed covariance in a csv file for later plotting and computation purposes.  
We also perform a sanity check: the covariance at the origin should equal the porosity of the image for the pore phase,
and 1-porosity for the grain phase.

In [7]:
covariance_orig_df = pd.DataFrame(direc_covariances_pore_phase_orig)
covariance_orig_df.to_csv(out_direc+"orig_pph.csv", delim=",", index=False)

covariances_orig_df_backload = pd.read_csv(out_direc+"orig_pph.csv")
covariances_orig_df_backload.head()

,x,y,z
0,0.126862,0.126862,0.126862
1,0.109027,0.111406,0.110832
2,0.094193,0.098307,0.097267
3,0.081682,0.087207,0.085761
4,0.071046,0.077693,0.075923


Due to the large size of these arrays we delete them manually before continuing to free up memory.

In [8]:
del two_point_covariance_pore_phase_orig
del two_point_direc

## Covariance Computation - Grain Phase

We perform the same computation as above, but exchange for the grain phase value of the image.

In [9]:
two_point_covariance_grain_phase_orig = {}
for i, direc in enumerate(["x", "y", "z"]):
    two_point_direc = two_point_correlation(orig_img, i, var=grain_phase)
    two_point_covariance_grain_phase_orig[direc] = two_point_direc

In [10]:
direc_covariances_grain_phase_orig = {}
for direc in ["x", "y", "z"]:
    direc_covariances_grain_phase_orig[direc] = np.mean(np.mean(two_point_covariance_grain_phase_orig[direc], axis=0), axis=0)
print direc_covariances_grain_phase_orig["x"].shape

(256L,)


In [11]:
covariance_orig_df = pd.DataFrame(direc_covariances_grain_phase_orig)
covariance_orig_df.to_csv(out_direc+"orig_gph.csv", delim=",", index=False)

covariances_orig_df_backload = pd.read_csv(out_direc+"orig_gph.csv")
covariances_orig_df_backload.head()

,x,y,z
0,0.873138,0.873138,0.873138
1,0.855445,0.857769,0.856924
2,0.840770,0.844745,0.843168
3,0.828417,0.833715,0.831490
4,0.817951,0.824284,0.821499


The covariance at the origin should be equal for all three directions and be equal to 1-porosity.

In [12]:
del two_point_covariance_grain_phase_orig
del two_point_direc

## Covariance Computation - Synthetic Samples - Pore Phase

We now perform above computation for the pore phase of the synthetic samples and store the results in csv files.

In [13]:
for seed in tnrange(seed_min, seed_max):
    im_in = tifffile.imread(synthetic_input+str(seed)+synthetic_ending)
    image = im_in.astype(np.int8)
    
    #determine phase values
    pore_phase = image.min()
    grain_phase = image.max()
    
    for phase, phase_label in zip([pore_phase, grain_phase], ["pph", "gph"]):
        #phase computation
        two_point_covariance = {}
        for i, direc in enumerate(["x", "y", "z"]):
            two_point_direc = two_point_correlation(image, i, var=phase)
            two_point_covariance[direc] = two_point_direc

        #phase averaging
        direc_covariances = {}
        for direc in ["x", "y", "z"]:
            direc_covariances[direc] = np.mean(np.mean(two_point_covariance[direc], axis=0), axis=0)

        #covariance storage
        covariance_df = pd.DataFrame(direc_covariances)
        covariance_df.to_csv(out_direc +"S_"+str(seed)+"_"+phase_label+".csv", delim=",", index=False)

We probably may shutdown the notebook after this, but to be sure we delete these, because they may hog all of the RAM.

In [14]:
del im_in
del image
del two_point_covariance
del direc_covariances
del covariance_df